# Getting Started with Sentinel-2

This notebook is a very simple example of the fundamentals of working with
Earth observation data using the Open Data Cube.

## Import libraries

In [ ]:
from datacube import Datacube
from odc.geo.geom import point

## Configure the environment

In [ ]:
dc = Datacube()

## Pick a study area

In [ ]:
# Find a location you're interested in on Google Maps and copy the coordinates
# by right-clicking on the map and clicking the coordinates

# These coords are in the order Y then X, or Latitude then Longitude
coords = -6.594, 106.793  # Bogor
aoi_point = point(coords[1], coords[0], crs="EPSG:4326")
bbox = aoi_point.buffer(0.1).boundingbox

landsat_stretch = dict(vmin=1000, vmax=4000)

datetime = "2025-03-01/2025-03-14"

# Preview the area
bbox.explore(zoom=8)

## Load data

This uses the Datacube library to handle loading of the actual data. The `dask_chunks` argument instructs the tool to use Dask
to lazy-load the data.

In [ ]:
data = dc.load(
    product="s2_l2a",
    measurements=["red", "green", "blue"],
    output_crs="EPSG:32750",
    resolution=10,
    time=("2025-03-01", "2025-03-14"),
    longitude=(bbox.left, bbox.right),
    latitude=(bbox.bottom, bbox.top),
    dask_chunks={},
    group_by="solar_day",
)

data

## Visualise data

This step uses `matplotlib` to view data as a static image. It takes a longer time to
run than previous steps, because it's actually loading the data to prepare the images.

The `to_array()` function is a trick used to be able to visualise the data as a
red, green, blue "true colour" image.

In [ ]:
data.to_array().plot.imshow(col="time", col_wrap=2, size=6, vmin=0, vmax=3000)

### Interactive map

This step uses another `odc` tool to visualise the data on a map.

In [ ]:
best = data.sel(time="2025-03-14").squeeze()
visualisation = best.odc.to_rgba(vmin=0, vmax=3000)

visualisation.odc.explore()

## Export data

Here we write data to disk, again using an `odc` tool.

In [ ]:
visualisation.odc.write_cog("sentinel2_example.tif", overwrite=True)